In [1]:
from tkinter.ttk import Combobox,Treeview,Label
import tkinter as tk
from PIL import ImageTk, Image
from tkinter import filedialog,messagebox,Label
from nltk.corpus import stopwords
import joblib
from nltk.tokenize import word_tokenize
import string
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import joblib
import swifter
import re

class GUI(tk.Tk):
    
    def __init__(self):
        super().__init__()
        self.title('Customer Satisfaction Analysis App')
        self.geometry('1600x900')

        #GAMABR
        img_analisis = ImageTk.PhotoImage(Image.open('1.png'))
        img_labelanalisis = ImageTk.PhotoImage(Image.open('2.png'))
        img = ImageTk.PhotoImage(Image.open('3.png'))
        imgtabel = ImageTk.PhotoImage(Image.open('4.png'))
        img_save = ImageTk.PhotoImage(Image.open('5.png'))
        img_upload = ImageTk.PhotoImage(Image.open('6.png'))

        #BACKGROUND
        self.label = tk.Label(image=img)
        self.label.image = img
        self.label.pack()

        #LABEL KATEGORI
        kl = tk.Label(self, text='Kategori :',bg="white" ,font=('helvetica',15,'bold'))
        kl.place(x=285, y=185)

        #COMBOBOX KATEGORI
        kategori = ['Fashion', 'Elektronik', 'Pertukangan']
        self.s = tk.StringVar()
        self.comboxkt = Combobox(self,values=kategori,textvariable=self.s,font= 12)
        self.comboxkt.place(x=285, y=210,width=850)

        #KOLOM ANALISIS ULASAN (entry)
        label_analisi = tk.Label(self, image=img_labelanalisis,bd=0)
        label_analisi.image = img_labelanalisis
        label_analisi.place(x=278,y=250)
        self.ulasan = tk.Entry(self, font=('helvetica',30),bd=0,highlightthickness=0,bg='#F6F7F6')
        self.ulasan.place(x=300, y=260,width=830, height=55)
        self.ulasan.insert(0, "Masukkan Ulasan")
        self.ulasan.configure(fg="#808080")
        self.ulasan.bind("<FocusIn>", self.watermark_entry)
        self.ulasan.bind("<FocusOut>", self.watermark_entry_out)

        #BUTTON ANALISI
        bt = tk.Button(self,image=img_analisis, command=self.analisis)
        bt.image = img_analisis
        bt.place(x=630, y=328,width=213,height=46)

        #LABEL TABEL
        label_tabel = tk.Label(self, image=imgtabel,bd=0)
        label_tabel.image = imgtabel
        label_tabel.place(x=245,y=385)

        #BUTTON UPLOAD FILE
        massal = tk.Button(self,image=img_upload, text='File Massal', command=self.analisis_csv)
        massal.image = img_upload
        massal.place(x=750, y=680,width=352,height=69)

        #BUTTON SAVE FILE
        savefile = tk.Button(self,image=img_save, command=self.export_to_csv)
        savefile.image = img_save
        savefile.place(x=380, y=680,width=352,height=69)

        self.previous_window = None

    #PLACEHOLDER / WATERMARK UNTUK KOLOM ENTRY ULASAN
    def watermark_entry(self, event):
        if self.ulasan.get() == "Masukkan Ulasan":
            self.ulasan.delete(0, "end")
            self.ulasan.configure(fg="#000000")

    def watermark_entry_out(self, event):
        if self.ulasan.get() == "":
            self.ulasan.insert(0, "Masukkan Ulasan")
            self.ulasan.configure(fg="#808080")
            
    def analisis(self):
        alay_dict = pd.read_csv('new_kamusalay.csv', header=None, encoding='latin-1')
        alay_dict = alay_dict.rename(columns={0: 'original',
                                                 1: 'replacement'})
        alay_dict.head()
            
        id_stopword_dict = pd.read_csv('stopwordbahasa.csv', header=None, encoding='latin-1')
        id_stopword_dict = id_stopword_dict.rename(columns={0: 'stopword'})

        factory = StemmerFactory()
        stemmer = factory.create_stemmer()

        slank_word_dict = {
        "brg": "barang",
        "krg": "kurang",
        "wrn": "warna",
        "kl": "kalau",
        "klo": "kalau",
        "dr": "dari",
        "yg": "yang",
        "tdk": "tidak",
        "dgn": "dengan",
        "tp": "tapi",
        "rapimantapcepatsampe": "rapi",
        "maaakkaaasssiiihhhhhhhhhhhhhh": "terima kasih",
        "makasihhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh": "terima kasih",
        "sipppsesuai": "sesuai",
        "responya": "respon",
        "thankyou": "terima kasih",
        "thx": "terima kasih",
        "mantaaaaaaaappp": "bagus",
        "deskripsirespon": "deskripsi",
        "cukuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuupp": "cukup",
        "josss": "bagus",
        "dcoba": "dicoba",
        "thnks": "terima kasih",
        "good": "bagus",
        "memuaskanrecomend": "memuaskan",
        "suuuuuuuuuuuuuuuuuuuuuudaaaaaaaah": "sudah",
        "diiiiteeeerimaaaaa": "diterima",
        "denggan": "dengan",
        "sampemksh": "sampai",
        "mantafff": "bagus",
        "matabgoodsesuai": "sesuai",
        "sampaithanks": "sampai",
        "goodcepat": "cepat",
        "makasihhhhh": "terima kasih",
        "trmksih": "terima kasih",
        "rapipokoknya": "rapi",
        "baguspengiriman": "bagus",
        "mantabcepat": "cepat",
        "gooooooooooooooooooood": "bagus",
        "deskripsilumayan": "lumayan",
        "sampaitapi": "sampai",
        "telitimohon": "teliti",
        "barangx": "barangnya",
        "cobamakasih": "terima kasih",
        "baguuuus": "bagus",
        "bagusbanget": "bagus",
        "memuaskanthanks": "memuaskan",
        "cepatpacking": "cepat",
        "bagusksksnsnjsjjsksmmsms": "bagus",
        "bagusssssssssssss": "bagus",
        "dinpake": "dipakai",
        "dehnext": "berikutnya",
        "berfungsimanteb": "berfungsi",
        "thanksthanksthanksthanks": "terima kasih",
        "sampe": "sampai",
        "smp": "sampai",
        "ntapsss": "keren",
        "keduakali": "kedua kali",
        "agak": "sedikit",
        "pas": "saat",
        "produkx": "produknya",
        "sukamkasih": "suka makasih",
        "gak": "tidak",
        "cpt": "cepat",
        "sdh": "sudah",
        "recommend": "rekomendasi",
        "bagusseller": "bagus",
        "bagusssss": "bagus",
        "bagussssss": "bagus",
        "baguuussss": "bagus",
        "cepatmakasih": "cepat",
        "cepatmantappp": "cepat",
        "cepatproduct": "cepat",
        "cepatrecomended": "cepat",
        "cepattoko": "cepat",
        "recomended": "rekomendasi",
        "recommended": "rekomendasi",
        "rekomendasi": "rekomendasi",
        "rekomended": "rekomendasi",
        "bgs": "bagus",
        "jgn": "jangan",
        "kereeen": "keren",
        "pake": "gunakan",
        "yaa": "ya",
        "ok": "baik",
        "tpi": "tapi",
        "bgt": "banget",
        "jg": "juga",
        "yg lain": "lainnya",
        "gan": "sobat",
        "sis": "kakak",
        "bener": "benar",
        "ngebut": "cepat",
        "mantul": "mantap",
        "nungguin": "menunggu",
        "langsung": "segera",
        "gpp": "tidak masalah",
        "wkwk": "hehe",
        "abis": "habis",
        "bkn": "bukan",
        "soalnya": "karena",
        "gini": "begini",
        "gmana": "bagaimana",
        "bgmn": "bagaimana",
        "nih": "ini",
        "gmn": "bagaimana",
        "dpt": "dapat",
        "jgn": "jangan",
        "bnyk": "banyak",
        "gtau": "tidak tahu",
        "jd": "jadi",
        "gmna": "bagaimana",
        "jelasin": "jelaskan",
        "ksh": "kasih",
        "hrus": "harus",
        "blg": "bilang",
        "tpis": "tetapi",
        "td": "tadi",
        "sdh": "sudah",
        "kemaren": "kemarin",
        "gt": "begitu",
        "omongin": "bicarakan",
        "klo": "kalau",
        "trz": "terus",
        "pada": "mereka",
        "ap": "apa",
        "d": "di",
        "k": "ke",
        "gtw": "gatau",
        "gk": "tidak",
        "lbh": "lebih",
        "gini": "begini",
        "aja": "saja",
        "lah": "lho",
        "jd": "jadi",
        "skrg": "sekarang",
        "gw": "saya",
        "lu": "kamu",
        "blg": "bilang",
        "mau": "ingin",
        "gmn": "bagaimana",
        }

        def lowercase(text):
            return text.lower()
        
        def remove_links(text):
                # menghapus tab, new line, ans back slice
            text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
            # menghapus non ASCII (emoticon, chinese word, .etc)
            text = text.encode('ascii', 'replace').decode('ascii')
            # menghapus mention, link, hashtag
            text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
            # menghapus URL
            return text.replace("http://", " ").replace("https://", " ")
        
        def remove_number(text):
            return  re.sub(r"\d+", " ", text) 

        def remove_punctuation(text):
            return text.translate(str.maketrans("","",string.punctuation))
        
        def remove_singl_char(text):
            return re.sub(r"\b[a-zA-Z]\b", " ", text)
        
        def slank_normalized_term(text):
            return ' '.join([slank_word_dict[word] if word in slank_word_dict else word for word in text.split(' ')])

        alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))

        def normalize_alay(text):
            return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])

        def remove_stopword(text):
            text = ' '.join([ ' ' if word in id_stopword_dict.stopword.values else word for word in text.split(' ')])
            text = re.sub('  +', ' ', text) # Remove extra spaces
            text = text.strip() # Remove space in begining and end of sentences
            return text

        # Mengubah kata menjadi kata dasar nya. Ex: Programmer, programming = program
        def stemming (text):
            return stemmer.stem(text)
        
        def preprocess(text):
            text = lowercase(text)
            text = remove_links(text)
            text = remove_number(text)
            text = remove_punctuation(text)
            text = remove_singl_char(text)
            text = slank_normalized_term(text)
            text = normalize_alay(text)
            text = remove_stopword(text)
            text = stemming(text)
            return text
        
        if self.comboxkt.get() =='Elektronik':
            with open('vocabele.pkl', 'rb') as file:
                    vokabuler_loaded = pickle.load(file)

            vectorizer_baru = TfidfVectorizer(vocabulary=vokabuler_loaded)

            teks_baru = self.ulasan.get()
            teks_baru = preprocess(teks_baru)
            print(teks_baru)
            teks_baru_transformed = vectorizer_baru.fit_transform([teks_baru])

            model = joblib.load('svm_resampling1_model_ele.sav')
            prediksi = model.predict(teks_baru_transformed)

            if prediksi == 2:
                prediksi = 'Puas'
            elif prediksi == 1:
                prediksi = 'Netral'
            elif prediksi == 0:
                prediksi = 'Tidak Puas'
                
            result_message = f"Hasil Analisis: {prediksi}"
            messagebox.showinfo("Analisis", result_message)

        elif self.comboxkt.get() == 'Fashion':
            with open('vocabfs.pkl', 'rb') as file:
                    vokabuler_loaded = pickle.load(file)

            vectorizer_baru = TfidfVectorizer(vocabulary=vokabuler_loaded)

            teks_baru = self.ulasan.get()
            teks_baru = preprocess(teks_baru)
            print(teks_baru)
            teks_baru_transformed = vectorizer_baru.fit_transform([teks_baru])

            model = joblib.load('rf_resampling1_model_fs.sav')
            prediksi = model.predict(teks_baru_transformed)

            if prediksi == 2:
                prediksi = 'Puas'
            elif prediksi == 1:
                prediksi = 'Netral'
            elif prediksi == 0:
                prediksi = 'Tidak Puas'
                
            result_message = f"Hasil Analisis: {prediksi}"
            messagebox.showinfo("Analisis", result_message)

        elif self.comboxkt.get() == 'Pertukangan':
            with open('vocabpt.pkl', 'rb') as file:
                    vokabuler_loaded = pickle.load(file)

            vectorizer_baru = TfidfVectorizer(vocabulary=vokabuler_loaded)

            teks_baru = self.ulasan.get()
            teks_baru = preprocess(teks_baru)
            print(teks_baru)
            teks_baru_transformed = vectorizer_baru.fit_transform([teks_baru])

            model = joblib.load('rf_resampling1_model_pt.sav')
            prediksi = model.predict(teks_baru_transformed)

            if prediksi == 2:
                prediksi = 'Puas'
            elif prediksi == 1:
                prediksi = 'Netral'
            elif prediksi == 0:
                prediksi = 'Tidak Puas'
                
            result_message = f"Hasil Analisis: {prediksi}"
            messagebox.showinfo("Analisis", result_message)

    def analisis_csv(self):
            
            def process_csv(file):
                global df1
                global tree
                df = pd.read_csv(file_path,delimiter=';', encoding='utf-8')
                    # Mencetak nama kolom yang ada di DataFrame
                print("Nama Kolom:", df.columns)
                
                # Pastikan kolom "ulasan" dan "label" ada di DataFrame df
                if "ulasan" in df.columns and "label" in df.columns:
                    df1 = df.head(10)
                    X = df1["ulasan"]
                    
                    # Memuat vokabuler yang telah disimpan
                    if self.comboxkt.get() == 'Elektronik':
                        with open('vocabele.pkl', 'rb') as file:
                            vokabuler_loaded = pickle.load(file)

                        vectorizer_baru = TfidfVectorizer(vocabulary=vokabuler_loaded)
                        teks_baru_transformed = vectorizer_baru.fit_transform(X)
                        model = joblib.load('svm_resampling1_model_ele.sav')
                        prediksi = model.predict(teks_baru_transformed)


                    elif self.comboxkt.get() == 'Fashion':
                        with open('vocabfs.pkl', 'rb') as file:
                            vokabuler_loaded = pickle.load(file)

                        vectorizer_baru = TfidfVectorizer(vocabulary=vokabuler_loaded)
                        teks_baru_transformed = vectorizer_baru.fit_transform(X)
                        model = joblib.load('rf_resampling1_model_fs.sav')
                        prediksi = model.predict(teks_baru_transformed)

                    elif self.comboxkt.get() == 'Pertukangan':
                        with open('vocabpt.pkl', 'rb') as file:
                            vokabuler_loaded = pickle.load(file)

                        vectorizer_baru = TfidfVectorizer(vocabulary=vokabuler_loaded)
                        teks_baru_transformed = vectorizer_baru.fit_transform(X)
                        model = joblib.load('rf_resampling1_model_pt.sav')
                        prediksi = model.predict(teks_baru_transformed)

                    df1["hasilanalisis"] = prediksi

                    def convert_to_label(numeric_label):
                        if numeric_label == 0:
                            return "Tidak Puas"
                        elif numeric_label == 1:
                            return "Netral"
                        elif numeric_label == 2:
                            return "Puas"
                        else:
                            return "Label tidak valid"

                    def display_dataframe():
                        for i, row in df1.iterrows():
                            label_converted = convert_to_label(row['hasilanalisis'])

                            tree.insert('', 'end', values=(row['ulasan'], label_converted))
                                    # Cetak nilai sebelum dimasukkan ke dalam TreeView

                    # Buat TreeView
                    columns = ('Ulasan', 'Hasil Analisis')
                    
                    tree = Treeview(self, columns=columns, show='headings')
                    tree.tag_configure('mytag')

                    # Atur lebar kolom
                    for col in columns:
                        if col == "Ulasan":
                            tree.heading(col, text=col)
                            tree.column(col, width=500)
                            continue
                        tree.heading(col, text=col)
                        tree.column(col, width=340)  # Ganti lebar sesuai kebutuhan
                    tree.place(x=307, y=418)

                    # Tambahkan data DataFrame ke dalam TreeView
                    display_dataframe()

                    for item in tree.get_children():
                        tree.item(item)
                        
                else:
                    messagebox.showerror("Error", "Kolom 'ulasan' atau 'label' tidak ditemukan dalam DataFrame.")

            global file_path
            file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
            if file_path:
                process_csv(file_path)    
    
    def export_to_csv(file):
        file_path_export = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])

        if file_path_export:
            df1.to_csv(file_path_export, index=False)
            messagebox.showinfo("Save Berhasil", "Data anda telah berhasil disimpan!")      

gui = GUI()
gui.mainloop()

2023-12-22 21:52:14.903 Python[96722:5347190] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


: 